In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import conj

# Importando a classe MetodoMatrizZ do código original
from src.zmatrix_steps import MetodoMatrizZdetailed

def exemplo_sistema_4_barras():
    # Definir o sistema
    barras = [1, 2, 3, 4]
    
    # Barra 1 é a slack (tipo 0)
    # Barras 2, 3 e 4 são PQ (tipo 2)
    barras_tipo = {1: 0, 2: 2, 3: 2, 4: 2}  # 0: slack, 1: PV, 2: PQ
    
    # Potências complexas (S = P + jQ) em pu
    # Nota: as cargas são negativas do ponto de vista da injeção de potência
    potencias = {
        1: 0.0 - 0.7j,  # Barra slack
        2: -1.28 - 1.28j,
        3: -0.32 - 0.16j,
        4: -1.6 - 0.80j
    }
    
    # Tensões iniciais (flat start com valores de 1.0 pu, exceto a barra slack)
    v_inicial = {
        1: 1.03 + 0.0j,  # Tensão de referência na barra slack
        2: 1.0 + 0.0j,
        3: 1.0 + 0.0j,
        4: 1.0 + 0.0j
    }
    
    # Linhas (conexões entre barras)
    linhas = [(1, 2), (2, 3), (2, 4)]
    
    # Calcular os elementos da matriz Y_bus
    
    # Impedâncias das linhas
    z12 = 0.0236 + 0.0233j
    z23 = 0.045 + 0.030j
    z24 = 0.0051 + 0.0005j
    
    # Admitâncias série das linhas
    y12 = 1 / z12
    y23 = 1 / z23
    y24 = 1 / z24
    
    # Admitância shunt total por linha (j0.01 pu)
    y_shunt = 0.01j
    
    # Construir a matriz de admitância Y_bus
    Y_bus = np.zeros((4, 4), dtype=complex)
    
    # Elementos diagonais (soma das admitâncias conectadas à barra)
    Y_bus[0, 0] = y12 + y_shunt/2  # Barra 1
    Y_bus[1, 1] = y12 + y23 + y24 + y_shunt/2 + y_shunt/2 + y_shunt/2  # Barra 2
    Y_bus[2, 2] = y23 + y_shunt/2  # Barra 3
    Y_bus[3, 3] = y24 + y_shunt/2  # Barra 4
    
    # Elementos não-diagonais (negativo da admitância entre as barras)
    Y_bus[0, 1] = Y_bus[1, 0] = -y12
    Y_bus[1, 2] = Y_bus[2, 1] = -y23
    Y_bus[1, 3] = Y_bus[3, 1] = -y24
    
    print("Matriz de Admitância Y_bus:")
    for i in range(4):
        row_str = ""
        for j in range(4):
            real = Y_bus[i, j].real
            imag = Y_bus[i, j].imag
            row_str += f"{real:.6f}{'+' if imag >= 0 else ''}{imag:.6f}j  "
        print(row_str)


        # Calcular e imprimir a matriz Z_bus (inversa da Y_bus reduzida)
    # Identifica a barra slack
    slack_bus = 1
    # Índices das barras não-slack
    barras_nao_slack = [b for b in barras if b != slack_bus]
    
    # Mapear índices originais para índices na matriz
    idx_map = {b: i for i, b in enumerate(barras)}
    idx_nao_slack = [idx_map[b] for b in barras_nao_slack]
    
    # Extrair a parte da Y_bus que corresponde às barras não-slack
    Y_reduzida = Y_bus[np.ix_(idx_nao_slack, idx_nao_slack)]
    
    # Calcular a matriz Z_bus (inversa da Y_bus reduzida)
    Z_bus = np.linalg.inv(Y_reduzida)
    
    print("\nMatriz de Impedância Z_bus (inversa da Y_bus reduzida):")
    print("Barras não-slack:", barras_nao_slack)
    for i, barra_i in enumerate(barras_nao_slack):
        row_str = f"Barra {barra_i}: "
        for j, barra_j in enumerate(barras_nao_slack):
            real = Z_bus[i, j].real
            imag = Z_bus[i, j].imag
            sign = "+" if imag >= 0 else ""
            row_str += f"Z{barra_i}{barra_j}={real:.6f}{sign}{imag:.6f}j  "
        print(row_str)
    
    # Tolerância conforme informado
    tolerancia = 0.0010
    
    # Criar instância do solver
    solver = MetodoMatrizZdetailed(Y_bus, barras, linhas, barras_tipo, potencias, v_inicial, tol=tolerancia)
    
    # Resolver usando método da matriz Z com substituição em bloco
    v_bloco, v_history_bloco, iteracoes_bloco = solver.substituicao_em_bloco()
    fluxos_bloco = solver.calcular_fluxos(v_bloco)
    
    # Exibir resultados
    solver.exibir_resultados(v_bloco, fluxos_bloco, "Substituição em Bloco", iteracoes_bloco)
    solver.plotar_convergencia(v_history_bloco, "Substituição em Bloco")
    
    # Resolver usando método da matriz Z com substituição direta
    v_direta, v_history_direta, iteracoes_direta = solver.substituicao_direta()
    fluxos_direta = solver.calcular_fluxos(v_direta)
    
    # Exibir resultados
    solver.exibir_resultados(v_direta, fluxos_direta, "Substituição Direta", iteracoes_direta)
    solver.plotar_convergencia(v_history_direta, "Substituição Direta")
    
    # Comparar número de iterações
    print(f"\nComparação do número de iterações:")
    print(f"Método de Substituição em Bloco: {iteracoes_bloco} iterações")
    print(f"Método de Substituição Direta: {iteracoes_direta} iterações")

if __name__ == "__main__":
    exemplo_sistema_4_barras()

ModuleNotFoundError: No module named 'src'